# User input data

In [ ]:
fdate = date(2019, 10, 20) # inserted by user
latitude = (47.7, 47.72)
longitude = (13.49, 13.52)
mask_tif = r'C:\Users\Emma\Desktop\ACube4Floods\code\mask_Strobler.tif'
target_names=['no_flood','flood']
path_out = '/home/boku/jupyter-datacube-docker-singularity/results'
identifier = f'ejemplo' #más adelante (en el acube hacer un identifier con el area y las fechas de las que sale el RF)
#identifier = f'ID_dates_{start_date}-{end_date}_area_{latitude}-{longitude}'
#num_images = 2 ##no lo uso pero lo puedo anyadir

In [ ]:
opt = 0  # # Visualization parameter

#### Aditional info

In [1]:
# # Flood event: Carinthia: 29102018
# #              Branau:    01062016
# #              Strobler:  27072019

# Libraries

In [1]:
import datacube
acube = datacube.Datacube(app='sample_9', env='acube')
import matplotlib.pyplot as plt
import math
import datetime
from datetime import timedelta, date
from datacube.utils.cog import write_cog
import gdal
from pylab import *
import numpy as np
import os
import matplotlib.colors as clr
import itertools
import csv

from acube_tools import query_data
from acube_tools import image_selection_byCloud
from acube_tools import VegetationIndex
from acube_tools import maskGeneration
from acube_tools import loadGeoTiff
from acube_tools import ind_VfoldCross
from acube_tools import RandomForestClassification
from acube_tools import plot_confusion_matrix
from acube_tools import rf_save_results
from acube_tools import diff_emma
from acube_tools import writeout
from acube_tools import scale
#from Tools import write_tiff
#from Tools import RGBplot

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score

/app/opt/venv/lib/python3.8/site-packages/datacube/drivers/postgres/_connections.py:81: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  EngineUrl(


NameError: name 'null' is not defined

# Code

In [ ]:
# Hardcoded variables
bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08',' B8A', 'B11', 'B12']
features = ['NDVI', 'NDWI', 'NDMI', 'TCARI', 'SAVI', 'KNDVI', 'MSAVI2']

In [ ]:
# Data after and before event:
day_b, s2_b, cloud_b, s2_geo_b = image_selection_byCloud (fdate, latitude, longitude, cpercen, 'before')
day_a, s2_a, cloud_a, s2_geo_a = image_selection_byCloud (fdate, latitude, longitude, cpercen, 'after')
print('Acube returns B02, B03, B04, B05, B06, B07, B08, B8A, B11 and B12 for Sentinel 2')
print('Shape sentinel 2 array before event:', s2_b.shape)
print('Shape sentinel 2 array after event:', s2_a.shape)

In [ ]:
'''# Mask: flood - no flood -- WARNING!!! I CAN NOT USE EMMA FUNCTION
if os.path.isfile(mask_tif):
    print('the mask is generated')
else:
    maskGeneration(s2_b, s2_geo_b, mask_tif)

clas = loadGeoTiff(mask_tif)
Y_total = np.reshape(clas, (clas.shape[0]*clas.shape[1], 1)).ravel()'''

In [ ]:
# Mask (change) -- WARNING!! TEMPORAL IT SHOULD BE CHANGED
clas = mask_tif
Y_total = np.reshape(clas, (clas.shape[0]*clas.shape[1], 1)).ravel()

In [ ]:
# Transform data to vector
s2_b_vector = s2_b.reshape(s2_b.shape[0], s2_b.shape[1]*s2_b.shape[2])
s2_b_vector = np.transpose(s2_b_vector)
s2_a_vector = s2_a.reshape(s2_a.shape[0], s2_a.shape[1]*s2_a.shape[2])
s2_a_vector = np.transpose(s2_a_vector)
print('Data vector shape before event:', s2_b_vector)
print('Data vector shape after event:', s2_a_vector)

In [ ]:
# Add vegetation indices
vi_b = VegetationIndex(s2_b_vector, bands, features)
vi_a = VegetationIndex(s2_a_vector, bands, features)

In [ ]:
# Concatenate data, get difference and clean --WARNING!!!!! No transform to bit????
data_b = np.concatenate((s2_b_vector, vi_b), axis=1)
data_a = np.concatenate((s2_a_vector, vi_a), axis=1)

In [ ]:
# Get difference
data = data_a - data_b
print('Data shape:', data.shape)

In [ ]:
# Clean all data (Nan): spectral, indices and mask
X_clean = data[~np.isnan(data).any(axis=1)]
Y_clean = Y_total[~np.isnan(data).any(axis=1)]

In [ ]:
# Select training data:
list_of_train = ind_VfoldCross(Y_clean, 1500) # WARNING!!! THINK HOW TO PRODUCE THIS NUMBER

X_train = X_clean[list_of_train, :]
Y_train = Y_clean[list_of_train]

In [ ]:
# Optimize and train the classifier:
Nt = [int(i) for i in np.round(np.linspace(50, 500, num=30))]
mtr = [int(i) for i in np.linspace(1, X_train.shape[1]-1, num=6)]

[cl_Final, param] = RandomForestClassification(X_train, Y_train,mtr, -1, 5, Nt)

In [ ]:
# Select test data:
list_of_test = diff_emma(range(Y_clean.shape[0]), list_of_train)

X_test = X_clean[list_of_test, :]
Y_test = Y_clean[list_of_test]

In [ ]:
# Predition in test samples:
Ypred = cl_Final.predict(X_test)

In [ ]:
# Statistics:

OA = accuracy_score(Y_test, Ypred)*100
print(OA)

kappa = cohen_kappa_score(Y_test, Ypred)
print(kappa)

CM = confusion_matrix(Y_test, Ypred)
print(CM)

OA1c = CM[1, 1] / (CM[1, 0] + CM[1, 1])
print(OA1c*100)
PEN1c = exp(-(CM[0, 1] / (-(CM[1, 0] + CM[1, 1]) / log(1 / 2))))
print(PEN1c)
# PEN1c = exp(-(CM[0, 1] / (-(sum(CM)) / log(1 / 2))))
SR1c = OA1c - (1 - PEN1c)

print(SR1c*100)

In [ ]:
# Save results RF --WARNING!! ADD SAVE RF CLASSIFIER
rf_save_results(CM, OA1c, kappa, PEN1c, SR1c, target_names, path_out, identifier)

In [ ]:
# Prediction map:
base_array[np.isnan(base_array)] = 999999999
Ypred = cl_Final.predict(base_array)
ProbPred = cl_Final.predict_proba(base_array)
Ypred[(base_array == 999999999).any(axis=1)] = 0
Y_clas = np.reshape(Ypred, (y1.shape[1], y1.shape[2]))

Prob_clas = np.reshape(ProbPred, (y1.shape[1], y1.shape[2], 2))

In [ ]:
# Scale the flood prob between 0 and 1:
Prob_flood_ns = Prob_clas[np.where(Y_clas == 1)[0], np.where(Y_clas == 1)[1], 1]
Prob_flood_ns = np.reshape(Prob_flood_ns, (Prob_flood_ns.shape[0], 1))
Prob_flood = scale(Prob_flood_ns)
Prob_fl = np.zeros((y1.shape[1], y1.shape[2]))

Prob_fl[np.where(Y_clas == 1)[0], np.where(Y_clas == 1)[1]] = np.reshape(Prob_flood[0], (Prob_flood[0].shape[0]))

In [ ]:
# Plotting classification and probablity results:
rcParams.update({'figure.autolayout': True})
colors1 = ['#000080', '#FFFFFF', '#FFFF00']
colors1 = [colors1[int(i-1)] for i in np.unique(Y_clas[~np.isnan(Y_clas)])]

cmap = clr.ListedColormap(colors1)

cmap = cm.get_cmap(cmap, len(np.unique(Y_clas[~np.isnan(Y_clas)])))

f = plt.figure()
plt.imshow(clas, cmap=cmap)
plt.axis('off')

f = plt.figure()
plt.imshow(Y_clas, cmap=cmap)
plt.axis('off')

f = plt.figure()
plt.imshow(Prob_fl)
plt.axis('off')
plt.show()

In [ ]:
writeout(clas, path_out, identifier, s2_geo_b['lon'], s2_geo_b['lat'])

In [ ]:
'''# Save the classification map in tiff:
tifsrc = gdal.Open(root_image)

if 0 == 1:
    write_tiff(clas, tifsrc, path_out, name_out)

    name = 'prediction.tif'
    write_tiff(Y_clas, tifsrc, path_out, name_out)

    name = 'probability.tif'
    write_tiff(Prob_fl, tifsrc, path_out, name_out)'''